# NiftiファイルからZ-scoreを自動抽出  
---  
  
引数：4D化した.niiファイルがあるフォルダまでのパス / 書き出すcsvの保存先 / mask名  
  
---  
  
入力：4D化した.niiファイルがあるフォルダまでのパス / 書き出すcsvの保存先 / mask名（chunks_list.csv, targets_list.csv）  
   
---
  
出力：all_raw.csv / raw_al45.csv / raw_al135.csv（chunks_list.csv, targets_list.csv）  
  
---  
  
注意：このプログラムはUbuntuでやったほうがいいかも（抽出する部位が多いほどデータが重くなってメモリがたりなくなる！）  
  

注意：このプログラムを実行する前に以下の作業をすること  

3D脳画像データ（前処理済みの.niiファイル）を4Dに変換  
  
1. SPM起動
2. Batchを開く
3. 画面左上SPM → Util → 3D to 4D File Conversion
3. 3D Volumesから4Dに変換したいファイル('swr'のついたniftiファイル全て)を選択
4. Output Filenameでファイル名を指定
5. 再生ボタンを押す

Motor mask（Z-score化したいボクセルの位置）の作成  
※ wfu_pickatlasはmatlabサーバーで実行（Macだと.nii形式で保存できなかった） 
※ 中山メモ：ubuntuでshareで作業するにはmatlabまで移動してshareを開く，基本的にsudoを使って作業（matlabもsudoで起動）    
 
1. wfu_pickatlasフォルダをダウンロード（ネットから or Shareかnakayamaフォルダのどっかにある）
2. MATLABでwfu_pickatlasフォルダのパスを通す
3. MATLABコマンドライン上で > wfu_pickatlas で起動
4. 画面左からマスクしたい部分を選択（全脳の場合は TD Hemispheres を全選択）
5. SAVE MASKからマスク画像をmaskとして保存（名前に'-'とか入れない）
6. SPM → fMRI起動
7. MenuからNormalize(Est & Wri)を選択
8. Bath Editorが立ち上がったらDataをダブルクリック
9. Image to Alignには'swr'がファイル名についているniftiファイルを適当に1つ選択
10. Image to Writeには先ほど作成したmaskファイルを選択
11. 再生ボタンを押す → wmask.niiができる
12. Batch Editorに戻りRealign(Reslice)を選択
13. Imagesをダブルクリック
14. 'swr'がファイル名についているniftiファイルを適当に一つ，先ほどできたwmask.niiファイルの順で選択
15. 再生ボタンを押す → rwmask.niiができる  


In [1]:
from mvpa2.suite import *
from mvpa2.datasets.mri import fmri_dataset
import os
import os.path
from os.path import join as pathjoin
from pprint import pprint
# from nifti import NiftiImage
import glob
import numpy as np
import pandas as pd
import sys
import pickle
# import dill
import csv

import nibabel as nib

Failed to import duecredit due to No module named 'duecredit'


 * Please note: warnings are printed only once, but underlying problem might occur many times *


/Users/nakayamarina/.pyenv/versions/anaconda3-4.4.0/lib/python3.6/site-packages/mvpa2/datasets/sources/skl_data.py:32: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  argnames, varargs, varkw, defaults = inspect.getargspec(fx)
<string>:60: DeprecationWarning: invalid escape sequence \#
<string>:59: DeprecationWarning: invalid escape sequence \#
<string>:59: DeprecationWarning: invalid escape sequence \#
<string>:65: DeprecationWarning: invalid escape sequence \c
/Users/nakayamarina/.pyenv/versions/anaconda3-4.4.0/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


コマンドライン引数で.niiファイルがあるディレクトリまでのパスを取得．

In [2]:
# !!!!!!!!!! scan_numらへん，dataset，zscoreのパラメータ書き換え


#args = sys.argv
#PATH = args[1]
#PATH_save = args[2]
#MASK_name = args[3]

# jupyter notebookのときはここで指定
PATH = '../../Data_mri/angledlinevid-1fe/20181119tm/'
PATH_save = '../MaskBrodmann/20181227rn/'
MASK_name = 'rwmaskBA.nii'

PATH_mask = PATH + MASK_name

# 前処理済みならswrがついた.niiファイルを選択
PATH_nii = PATH + '4D.nii'



In [3]:
# RawDataのディレクトリ名・パス
DIR_RAW = PATH_save + 'RawData'
PATH_RAW = DIR_RAW + '/'

# すでに存在する場合は何もせず，存在していない場合はディレクトリ作成
if not os.path.exists(DIR_RAW):
    os.mkdir(DIR_RAW)

FileNotFoundError: [Errno 2] No such file or directory: '../MaskBrodmann2/20181119tm/RawData'

In [5]:
headcoil = PATH.split('/')[5]


# 総スキャン数
scan_num = 200

# restのスキャン数
restNum = 8

# 1タスクのスキャン数
taskNum = 88

# 試行数
runNum = 4


## splitTaks関数

全てのZ-scoreをまとめたものdataで受け取り，Rock時とscissor時とpaper時のデータを分けてcsvファイルで書き出し

In [10]:
def splitTask(brain):
    
    # 各タスクごとのマスク作成
    maskAl45 = ([False] * restNum) + ([False] * taskNum) + ([False] * restNum) + ([True] * taskNum) + ([False] * restNum) + ([False] * restNum) + ([True] * taskNum) + ([False] * restNum) + ([False] * taskNum) + ([False] * restNum) + ([False] * restNum) + ([False] * taskNum) + ([False] * restNum) + ([True] * taskNum) + ([False] * restNum) + ([False] * restNum) + ([False] * taskNum) + ([False] * restNum) + ([True] * taskNum) + ([False] * restNum)
                
    maskAl135 = ([False] * restNum) + ([True] * taskNum) + ([False] * restNum) + ([False] * taskNum) + ([False] * restNum) + ([False] * restNum) + ([False] * taskNum) + ([False] * restNum) + ([True] * taskNum) + ([False] * restNum) + ([False] * restNum) + ([True] * taskNum) + ([False] * restNum) + ([False] * taskNum) + ([False] * restNum) + ([False] * restNum) + ([True] * taskNum) + ([False] * restNum) + ([False] * taskNum) + ([False] * restNum)

    # mask適用
    dataAl45 = brain[maskAl45]
    dataAl135 = brain[maskAl135]

    # splitVoxRun関数で各ボクセル，各試行で分割，csv書き出し
    vox_run_al45 = splitVoxRun(dataAl45)
    al45_file = PATH_RAW + 'raw_al45.csv'
    vox_run_al45.to_csv(al45_file)
    print(al45_file)
    
    vox_run_al135 = splitVoxRun(dataAl135)
    al135_file = PATH_RAW + 'raw_al135.csv'
    vox_run_al135.to_csv(al135_file)
    print(al135_file)

SyntaxError: invalid syntax (<ipython-input-10-3cc47e5a03d7>, line 4)

In [58]:
def splitVoxRun(data):

    # 各試行ごとに分割，横結合（Voxel1-Run1, Voxel1-Run2, Voxel1-Run3, Voxel1-Run4, Voxel2-Run1, Voxel2-Run2...）

    # データ格納用
    vox_run_all = pd.DataFrame(index = [], columns = [])

    for i in range(len(data.columns)):

        print('----- Voxel' + str(i + 1) + ' -----')
        print(len(list(data.iloc[:,i])))

        # ボクセルで試行ごとに分割，reshapeを使って1列データを(試行数，1タスクのスキャン数)に
        vox_run = np.reshape(list(data.iloc[:, i]), (runNum, taskNum))

        # 転置してデータフレーム化
        vox_run = pd.DataFrame(vox_run).T

        # 列名つける
        col_name = ['Voxel' + str(i + 1)] * runNum
        vox_run.columns = col_name

        # データ格納
        vox_run_all = pd.concat([vox_run_all, vox_run], axis = 1)

    return vox_run_all


# main関数

* fMRIデータ（niftiファイル）読み込み
* 全ボクセルデータZ-score化，抽出，書きだし

In [37]:
if __name__ == '__main__':

    # 4D化した.niiファイル名リストを作成

    nifti = [PATH_nii]


In [8]:
    # 教師データの作成（この作業なしにやる方法がわからないので，必要のない作業ではあるがやる）

    task_p1 = ['-1'] * restNum
    task_p2 = ['-1'] * restNum
    
    task45 = ['0'] * taskNum
    task135 = ['1'] * taskNum
    taskRest = ['-1'] * restNum

    # pattern1:Rest -> 45 -> Rest -> 135 -> Rest
    task_p1.extend(task45)
    task_p1.extend(taskRest)
    task_p1.extend(task135)
    task_p1.extend(taskRest)
    
    # pattern2:Rest -> 135 -> Rest -> 45 -> Rest
    task_p2.extend(task135)
    task_p2.extend(taskRest)
    task_p2.extend(task45)
    task_p2.extend(taskRest)
    
    task = task_p2 + task_p1 + task_p2 + task_p2
    
    target = pd.DataFrame(task)

    PATH_target = PATH + 'targets_list.csv'
    target.to_csv(PATH_target, index = False, header = None)
    print('target')

    targets_list = []

    targets_file = open(PATH_target, 'rU')
    dataReader = csv.reader(targets_file)

    for row in dataReader:
        targets_list.append(row[0])

target


/Users/nakayamarina/.pyenv/versions/anaconda3-4.4.0/lib/python3.6/site-packages/ipykernel_launcher.py:32: DeprecationWarning: 'U' mode is deprecated


In [44]:
    # チャンク（試行数リスト？）の作成（この作業なしにやる方法がわからないので，必要のない作業ではあるがやる）

    chunk = ['1'] * scan_num
    chunk0 = ['2'] * scan_num
    chunk1 = ['3'] * scan_num
    chunk2 = ['4'] * scan_num
    
    chunk.extend(chunk0)
    chunk.extend(chunk1)
    chunk.extend(chunk2)

    chunks = pd.DataFrame(chunk)

    PATH_chunk = PATH + 'chunks_list.csv'
    chunks.to_csv(PATH_chunk, index = False, header = None)
    print('chunks')

    chunks_list = []

    chunks = open(PATH_chunk, 'rU')

    for x in chunks:
        chunks_list.append(x.rstrip('\r\n'))

    chunks.close()


chunks


/Users/nakayamarina/.pyenv/versions/anaconda3-4.4.0/lib/python3.6/site-packages/ipykernel_launcher.py:20: DeprecationWarning: 'U' mode is deprecated


In [52]:
    # データセットの整形

    dataset = fmri_dataset(nifti, targets=targets_list, chunks=chunks_list, mask=PATH_mask ,sprefix='voxel', tprefix='time', add_fa=None)

    print('dataset ready')
    
    poly_detrend(dataset, polyord=1, chunks_attr='chunks')

    dataset = dataset[np.array([l in ['-1', '0', '1']
                               for l in dataset.targets], dtype='bool')]

dataset ready


In [53]:
    # Z-score抽出 → データフレーム化
    zscore(dataset, chunks_attr='chunks', param_est=('targets', ['0', '1']), dtype='float32')
    print('z-score')

    VoxelData = pd.DataFrame(np.array(dataset[:,:]))
    print(VoxelData .shape)

z-score
(800, 68255)


In [54]:
    voxName = []
    
    for i in range(len(VoxelData.columns)):
        
        name = 'Voxel' + str(i+1)
        print(name)
        
        voxName.append(name)
    
    VoxelData.columns = voxName

Voxel1
Voxel2
Voxel3
Voxel4
Voxel5
Voxel6
Voxel7
Voxel8
Voxel9
Voxel10
Voxel11
Voxel12
Voxel13
Voxel14
Voxel15
Voxel16
Voxel17
Voxel18
Voxel19
Voxel20
Voxel21
Voxel22
Voxel23
Voxel24
Voxel25
Voxel26
Voxel27
Voxel28
Voxel29
Voxel30
Voxel31
Voxel32
Voxel33
Voxel34
Voxel35
Voxel36
Voxel37
Voxel38
Voxel39
Voxel40
Voxel41
Voxel42
Voxel43
Voxel44
Voxel45
Voxel46
Voxel47
Voxel48
Voxel49
Voxel50
Voxel51
Voxel52
Voxel53
Voxel54
Voxel55
Voxel56
Voxel57
Voxel58
Voxel59
Voxel60
Voxel61
Voxel62
Voxel63
Voxel64
Voxel65
Voxel66
Voxel67
Voxel68
Voxel69
Voxel70
Voxel71
Voxel72
Voxel73
Voxel74
Voxel75
Voxel76
Voxel77
Voxel78
Voxel79
Voxel80
Voxel81
Voxel82
Voxel83
Voxel84
Voxel85
Voxel86
Voxel87
Voxel88
Voxel89
Voxel90
Voxel91
Voxel92
Voxel93
Voxel94
Voxel95
Voxel96
Voxel97
Voxel98
Voxel99
Voxel100
Voxel101
Voxel102
Voxel103
Voxel104
Voxel105
Voxel106
Voxel107
Voxel108
Voxel109
Voxel110
Voxel111
Voxel112
Voxel113
Voxel114
Voxel115
Voxel116
Voxel117
Voxel118
Voxel119
Voxel120
Voxel121
Voxel122
Voxel123
V

Voxel1427
Voxel1428
Voxel1429
Voxel1430
Voxel1431
Voxel1432
Voxel1433
Voxel1434
Voxel1435
Voxel1436
Voxel1437
Voxel1438
Voxel1439
Voxel1440
Voxel1441
Voxel1442
Voxel1443
Voxel1444
Voxel1445
Voxel1446
Voxel1447
Voxel1448
Voxel1449
Voxel1450
Voxel1451
Voxel1452
Voxel1453
Voxel1454
Voxel1455
Voxel1456
Voxel1457
Voxel1458
Voxel1459
Voxel1460
Voxel1461
Voxel1462
Voxel1463
Voxel1464
Voxel1465
Voxel1466
Voxel1467
Voxel1468
Voxel1469
Voxel1470
Voxel1471
Voxel1472
Voxel1473
Voxel1474
Voxel1475
Voxel1476
Voxel1477
Voxel1478
Voxel1479
Voxel1480
Voxel1481
Voxel1482
Voxel1483
Voxel1484
Voxel1485
Voxel1486
Voxel1487
Voxel1488
Voxel1489
Voxel1490
Voxel1491
Voxel1492
Voxel1493
Voxel1494
Voxel1495
Voxel1496
Voxel1497
Voxel1498
Voxel1499
Voxel1500
Voxel1501
Voxel1502
Voxel1503
Voxel1504
Voxel1505
Voxel1506
Voxel1507
Voxel1508
Voxel1509
Voxel1510
Voxel1511
Voxel1512
Voxel1513
Voxel1514
Voxel1515
Voxel1516
Voxel1517
Voxel1518
Voxel1519
Voxel1520
Voxel1521
Voxel1522
Voxel1523
Voxel1524
Voxel1525
Voxel1526


Voxel2849
Voxel2850
Voxel2851
Voxel2852
Voxel2853
Voxel2854
Voxel2855
Voxel2856
Voxel2857
Voxel2858
Voxel2859
Voxel2860
Voxel2861
Voxel2862
Voxel2863
Voxel2864
Voxel2865
Voxel2866
Voxel2867
Voxel2868
Voxel2869
Voxel2870
Voxel2871
Voxel2872
Voxel2873
Voxel2874
Voxel2875
Voxel2876
Voxel2877
Voxel2878
Voxel2879
Voxel2880
Voxel2881
Voxel2882
Voxel2883
Voxel2884
Voxel2885
Voxel2886
Voxel2887
Voxel2888
Voxel2889
Voxel2890
Voxel2891
Voxel2892
Voxel2893
Voxel2894
Voxel2895
Voxel2896
Voxel2897
Voxel2898
Voxel2899
Voxel2900
Voxel2901
Voxel2902
Voxel2903
Voxel2904
Voxel2905
Voxel2906
Voxel2907
Voxel2908
Voxel2909
Voxel2910
Voxel2911
Voxel2912
Voxel2913
Voxel2914
Voxel2915
Voxel2916
Voxel2917
Voxel2918
Voxel2919
Voxel2920
Voxel2921
Voxel2922
Voxel2923
Voxel2924
Voxel2925
Voxel2926
Voxel2927
Voxel2928
Voxel2929
Voxel2930
Voxel2931
Voxel2932
Voxel2933
Voxel2934
Voxel2935
Voxel2936
Voxel2937
Voxel2938
Voxel2939
Voxel2940
Voxel2941
Voxel2942
Voxel2943
Voxel2944
Voxel2945
Voxel2946
Voxel2947
Voxel2948


Voxel4470
Voxel4471
Voxel4472
Voxel4473
Voxel4474
Voxel4475
Voxel4476
Voxel4477
Voxel4478
Voxel4479
Voxel4480
Voxel4481
Voxel4482
Voxel4483
Voxel4484
Voxel4485
Voxel4486
Voxel4487
Voxel4488
Voxel4489
Voxel4490
Voxel4491
Voxel4492
Voxel4493
Voxel4494
Voxel4495
Voxel4496
Voxel4497
Voxel4498
Voxel4499
Voxel4500
Voxel4501
Voxel4502
Voxel4503
Voxel4504
Voxel4505
Voxel4506
Voxel4507
Voxel4508
Voxel4509
Voxel4510
Voxel4511
Voxel4512
Voxel4513
Voxel4514
Voxel4515
Voxel4516
Voxel4517
Voxel4518
Voxel4519
Voxel4520
Voxel4521
Voxel4522
Voxel4523
Voxel4524
Voxel4525
Voxel4526
Voxel4527
Voxel4528
Voxel4529
Voxel4530
Voxel4531
Voxel4532
Voxel4533
Voxel4534
Voxel4535
Voxel4536
Voxel4537
Voxel4538
Voxel4539
Voxel4540
Voxel4541
Voxel4542
Voxel4543
Voxel4544
Voxel4545
Voxel4546
Voxel4547
Voxel4548
Voxel4549
Voxel4550
Voxel4551
Voxel4552
Voxel4553
Voxel4554
Voxel4555
Voxel4556
Voxel4557
Voxel4558
Voxel4559
Voxel4560
Voxel4561
Voxel4562
Voxel4563
Voxel4564
Voxel4565
Voxel4566
Voxel4567
Voxel4568
Voxel4569


Voxel6048
Voxel6049
Voxel6050
Voxel6051
Voxel6052
Voxel6053
Voxel6054
Voxel6055
Voxel6056
Voxel6057
Voxel6058
Voxel6059
Voxel6060
Voxel6061
Voxel6062
Voxel6063
Voxel6064
Voxel6065
Voxel6066
Voxel6067
Voxel6068
Voxel6069
Voxel6070
Voxel6071
Voxel6072
Voxel6073
Voxel6074
Voxel6075
Voxel6076
Voxel6077
Voxel6078
Voxel6079
Voxel6080
Voxel6081
Voxel6082
Voxel6083
Voxel6084
Voxel6085
Voxel6086
Voxel6087
Voxel6088
Voxel6089
Voxel6090
Voxel6091
Voxel6092
Voxel6093
Voxel6094
Voxel6095
Voxel6096
Voxel6097
Voxel6098
Voxel6099
Voxel6100
Voxel6101
Voxel6102
Voxel6103
Voxel6104
Voxel6105
Voxel6106
Voxel6107
Voxel6108
Voxel6109
Voxel6110
Voxel6111
Voxel6112
Voxel6113
Voxel6114
Voxel6115
Voxel6116
Voxel6117
Voxel6118
Voxel6119
Voxel6120
Voxel6121
Voxel6122
Voxel6123
Voxel6124
Voxel6125
Voxel6126
Voxel6127
Voxel6128
Voxel6129
Voxel6130
Voxel6131
Voxel6132
Voxel6133
Voxel6134
Voxel6135
Voxel6136
Voxel6137
Voxel6138
Voxel6139
Voxel6140
Voxel6141
Voxel6142
Voxel6143
Voxel6144
Voxel6145
Voxel6146
Voxel6147


...

Voxel60673
Voxel60674
Voxel60675
Voxel60676
Voxel60677
Voxel60678
Voxel60679
Voxel60680
Voxel60681
Voxel60682
Voxel60683
Voxel60684
Voxel60685
Voxel60686
Voxel60687
Voxel60688
Voxel60689
Voxel60690
Voxel60691
Voxel60692
Voxel60693
Voxel60694
Voxel60695
Voxel60696
Voxel60697
Voxel60698
Voxel60699
Voxel60700
Voxel60701
Voxel60702
Voxel60703
Voxel60704
Voxel60705
Voxel60706
Voxel60707
Voxel60708
Voxel60709
Voxel60710
Voxel60711
Voxel60712
Voxel60713
Voxel60714
Voxel60715
Voxel60716
Voxel60717
Voxel60718
Voxel60719
Voxel60720
Voxel60721
Voxel60722
Voxel60723
Voxel60724
Voxel60725
Voxel60726
Voxel60727
Voxel60728
Voxel60729
Voxel60730
Voxel60731
Voxel60732
Voxel60733
Voxel60734
Voxel60735
Voxel60736
Voxel60737
Voxel60738
Voxel60739
Voxel60740
Voxel60741
Voxel60742
Voxel60743
Voxel60744
Voxel60745
Voxel60746
Voxel60747
Voxel60748
Voxel60749
Voxel60750
Voxel60751
Voxel60752
Voxel60753
Voxel60754
Voxel60755
Voxel60756
Voxel60757
Voxel60758
Voxel60759
Voxel60760
Voxel60761
Voxel60762
Voxel60763

Voxel62278
Voxel62279
Voxel62280
Voxel62281
Voxel62282
Voxel62283
Voxel62284
Voxel62285
Voxel62286
Voxel62287
Voxel62288
Voxel62289
Voxel62290
Voxel62291
Voxel62292
Voxel62293
Voxel62294
Voxel62295
Voxel62296
Voxel62297
Voxel62298
Voxel62299
Voxel62300
Voxel62301
Voxel62302
Voxel62303
Voxel62304
Voxel62305
Voxel62306
Voxel62307
Voxel62308
Voxel62309
Voxel62310
Voxel62311
Voxel62312
Voxel62313
Voxel62314
Voxel62315
Voxel62316
Voxel62317
Voxel62318
Voxel62319
Voxel62320
Voxel62321
Voxel62322
Voxel62323
Voxel62324
Voxel62325
Voxel62326
Voxel62327
Voxel62328
Voxel62329
Voxel62330
Voxel62331
Voxel62332
Voxel62333
Voxel62334
Voxel62335
Voxel62336
Voxel62337
Voxel62338
Voxel62339
Voxel62340
Voxel62341
Voxel62342
Voxel62343
Voxel62344
Voxel62345
Voxel62346
Voxel62347
Voxel62348
Voxel62349
Voxel62350
Voxel62351
Voxel62352
Voxel62353
Voxel62354
Voxel62355
Voxel62356
Voxel62357
Voxel62358
Voxel62359
Voxel62360
Voxel62361
Voxel62362
Voxel62363
Voxel62364
Voxel62365
Voxel62366
Voxel62367
Voxel62368

Voxel63878
Voxel63879
Voxel63880
Voxel63881
Voxel63882
Voxel63883
Voxel63884
Voxel63885
Voxel63886
Voxel63887
Voxel63888
Voxel63889
Voxel63890
Voxel63891
Voxel63892
Voxel63893
Voxel63894
Voxel63895
Voxel63896
Voxel63897
Voxel63898
Voxel63899
Voxel63900
Voxel63901
Voxel63902
Voxel63903
Voxel63904
Voxel63905
Voxel63906
Voxel63907
Voxel63908
Voxel63909
Voxel63910
Voxel63911
Voxel63912
Voxel63913
Voxel63914
Voxel63915
Voxel63916
Voxel63917
Voxel63918
Voxel63919
Voxel63920
Voxel63921
Voxel63922
Voxel63923
Voxel63924
Voxel63925
Voxel63926
Voxel63927
Voxel63928
Voxel63929
Voxel63930
Voxel63931
Voxel63932
Voxel63933
Voxel63934
Voxel63935
Voxel63936
Voxel63937
Voxel63938
Voxel63939
Voxel63940
Voxel63941
Voxel63942
Voxel63943
Voxel63944
Voxel63945
Voxel63946
Voxel63947
Voxel63948
Voxel63949
Voxel63950
Voxel63951
Voxel63952
Voxel63953
Voxel63954
Voxel63955
Voxel63956
Voxel63957
Voxel63958
Voxel63959
Voxel63960
Voxel63961
Voxel63962
Voxel63963
Voxel63964
Voxel63965
Voxel63966
Voxel63967
Voxel63968

Voxel65511
Voxel65512
Voxel65513
Voxel65514
Voxel65515
Voxel65516
Voxel65517
Voxel65518
Voxel65519
Voxel65520
Voxel65521
Voxel65522
Voxel65523
Voxel65524
Voxel65525
Voxel65526
Voxel65527
Voxel65528
Voxel65529
Voxel65530
Voxel65531
Voxel65532
Voxel65533
Voxel65534
Voxel65535
Voxel65536
Voxel65537
Voxel65538
Voxel65539
Voxel65540
Voxel65541
Voxel65542
Voxel65543
Voxel65544
Voxel65545
Voxel65546
Voxel65547
Voxel65548
Voxel65549
Voxel65550
Voxel65551
Voxel65552
Voxel65553
Voxel65554
Voxel65555
Voxel65556
Voxel65557
Voxel65558
Voxel65559
Voxel65560
Voxel65561
Voxel65562
Voxel65563
Voxel65564
Voxel65565
Voxel65566
Voxel65567
Voxel65568
Voxel65569
Voxel65570
Voxel65571
Voxel65572
Voxel65573
Voxel65574
Voxel65575
Voxel65576
Voxel65577
Voxel65578
Voxel65579
Voxel65580
Voxel65581
Voxel65582
Voxel65583
Voxel65584
Voxel65585
Voxel65586
Voxel65587
Voxel65588
Voxel65589
Voxel65590
Voxel65591
Voxel65592
Voxel65593
Voxel65594
Voxel65595
Voxel65596
Voxel65597
Voxel65598
Voxel65599
Voxel65600
Voxel65601

Voxel67086
Voxel67087
Voxel67088
Voxel67089
Voxel67090
Voxel67091
Voxel67092
Voxel67093
Voxel67094
Voxel67095
Voxel67096
Voxel67097
Voxel67098
Voxel67099
Voxel67100
Voxel67101
Voxel67102
Voxel67103
Voxel67104
Voxel67105
Voxel67106
Voxel67107
Voxel67108
Voxel67109
Voxel67110
Voxel67111
Voxel67112
Voxel67113
Voxel67114
Voxel67115
Voxel67116
Voxel67117
Voxel67118
Voxel67119
Voxel67120
Voxel67121
Voxel67122
Voxel67123
Voxel67124
Voxel67125
Voxel67126
Voxel67127
Voxel67128
Voxel67129
Voxel67130
Voxel67131
Voxel67132
Voxel67133
Voxel67134
Voxel67135
Voxel67136
Voxel67137
Voxel67138
Voxel67139
Voxel67140
Voxel67141
Voxel67142
Voxel67143
Voxel67144
Voxel67145
Voxel67146
Voxel67147
Voxel67148
Voxel67149
Voxel67150
Voxel67151
Voxel67152
Voxel67153
Voxel67154
Voxel67155
Voxel67156
Voxel67157
Voxel67158
Voxel67159
Voxel67160
Voxel67161
Voxel67162
Voxel67163
Voxel67164
Voxel67165
Voxel67166
Voxel67167
Voxel67168
Voxel67169
Voxel67170
Voxel67171
Voxel67172
Voxel67173
Voxel67174
Voxel67175
Voxel67176

In [24]:
    # 全データのcevファイル書き出し
    PATH_all = PATH_RAW + 'all_raw.csv'
    VoxelData.to_csv(PATH_all, index = False)
    print(PATH_all)

../State-2fe_MaskMotor/20181029rn/64ch/RawData/all_raw.csv


In [60]:
    splitTask(VoxelData)

----- Voxel1 -----
----- Voxel2 -----
----- Voxel3 -----
----- Voxel4 -----
----- Voxel5 -----
----- Voxel6 -----
----- Voxel7 -----
----- Voxel8 -----
----- Voxel9 -----
----- Voxel10 -----
----- Voxel11 -----
----- Voxel12 -----


/Users/nakayamarina/.pyenv/versions/anaconda3-4.4.0/lib/python3.6/site-packages/numpy/core/fromnumeric.py:57: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  return getattr(obj, method)(*args, **kwds)


----- Voxel13 -----
----- Voxel14 -----
----- Voxel15 -----
----- Voxel16 -----
----- Voxel17 -----
----- Voxel18 -----
----- Voxel19 -----
----- Voxel20 -----
----- Voxel21 -----
----- Voxel22 -----
----- Voxel23 -----
----- Voxel24 -----
----- Voxel25 -----
----- Voxel26 -----
----- Voxel27 -----
----- Voxel28 -----
----- Voxel29 -----
----- Voxel30 -----
----- Voxel31 -----
----- Voxel32 -----
----- Voxel33 -----
----- Voxel34 -----
----- Voxel35 -----
----- Voxel36 -----
----- Voxel37 -----
----- Voxel38 -----
----- Voxel39 -----
----- Voxel40 -----
----- Voxel41 -----
----- Voxel42 -----
----- Voxel43 -----
----- Voxel44 -----
----- Voxel45 -----
----- Voxel46 -----
----- Voxel47 -----
----- Voxel48 -----
----- Voxel49 -----
----- Voxel50 -----
----- Voxel51 -----
----- Voxel52 -----
----- Voxel53 -----
----- Voxel54 -----
----- Voxel55 -----
----- Voxel56 -----
----- Voxel57 -----
----- Voxel58 -----
----- Voxel59 -----
----- Voxel60 -----
----- Voxel61 -----
----- Voxel62 -----


----- Voxel444 -----
----- Voxel445 -----
----- Voxel446 -----
----- Voxel447 -----
----- Voxel448 -----
----- Voxel449 -----
----- Voxel450 -----
----- Voxel451 -----
----- Voxel452 -----
----- Voxel453 -----
----- Voxel454 -----
----- Voxel455 -----
----- Voxel456 -----
----- Voxel457 -----
----- Voxel458 -----
----- Voxel459 -----
----- Voxel460 -----
----- Voxel461 -----
----- Voxel462 -----
----- Voxel463 -----
----- Voxel464 -----
----- Voxel465 -----
----- Voxel466 -----
----- Voxel467 -----
----- Voxel468 -----
----- Voxel469 -----
----- Voxel470 -----
----- Voxel471 -----
----- Voxel472 -----
----- Voxel473 -----
----- Voxel474 -----
----- Voxel475 -----
----- Voxel476 -----
----- Voxel477 -----
----- Voxel478 -----
----- Voxel479 -----
----- Voxel480 -----
----- Voxel481 -----
----- Voxel482 -----
----- Voxel483 -----
----- Voxel484 -----
----- Voxel485 -----
----- Voxel486 -----
----- Voxel487 -----
----- Voxel488 -----
----- Voxel489 -----
----- Voxel490 -----
----- Voxel49

----- Voxel840 -----
----- Voxel841 -----
----- Voxel842 -----
----- Voxel843 -----
----- Voxel844 -----
----- Voxel845 -----
----- Voxel846 -----
----- Voxel847 -----
----- Voxel848 -----
----- Voxel849 -----
----- Voxel850 -----
----- Voxel851 -----
----- Voxel852 -----
----- Voxel853 -----
----- Voxel854 -----
----- Voxel855 -----
----- Voxel856 -----
----- Voxel857 -----
----- Voxel858 -----
----- Voxel859 -----
----- Voxel860 -----
----- Voxel861 -----
----- Voxel862 -----
----- Voxel863 -----
----- Voxel864 -----
----- Voxel865 -----
----- Voxel866 -----
----- Voxel867 -----
----- Voxel868 -----
----- Voxel869 -----
----- Voxel870 -----
----- Voxel871 -----
----- Voxel872 -----
----- Voxel873 -----
----- Voxel874 -----
----- Voxel875 -----
----- Voxel876 -----
----- Voxel877 -----
----- Voxel878 -----
----- Voxel879 -----
----- Voxel880 -----
----- Voxel881 -----
----- Voxel882 -----
----- Voxel883 -----
----- Voxel884 -----
----- Voxel885 -----
----- Voxel886 -----
----- Voxel88

...

----- Voxel7259 -----
----- Voxel7260 -----
----- Voxel7261 -----
----- Voxel7262 -----
----- Voxel7263 -----
----- Voxel7264 -----
----- Voxel7265 -----
----- Voxel7266 -----
----- Voxel7267 -----
----- Voxel7268 -----
----- Voxel7269 -----
----- Voxel7270 -----
----- Voxel7271 -----
----- Voxel7272 -----
----- Voxel7273 -----
----- Voxel7274 -----
----- Voxel7275 -----
----- Voxel7276 -----
----- Voxel7277 -----
----- Voxel7278 -----
----- Voxel7279 -----
----- Voxel7280 -----
----- Voxel7281 -----
----- Voxel7282 -----
----- Voxel7283 -----
----- Voxel7284 -----
----- Voxel7285 -----
----- Voxel7286 -----
----- Voxel7287 -----
----- Voxel7288 -----
----- Voxel7289 -----
----- Voxel7290 -----
----- Voxel7291 -----
----- Voxel7292 -----
----- Voxel7293 -----
----- Voxel7294 -----
----- Voxel7295 -----
----- Voxel7296 -----
----- Voxel7297 -----
----- Voxel7298 -----
----- Voxel7299 -----
----- Voxel7300 -----
----- Voxel7301 -----
----- Voxel7302 -----
----- Voxel7303 -----
----- Voxe

----- Voxel7632 -----
----- Voxel7633 -----
----- Voxel7634 -----
----- Voxel7635 -----
----- Voxel7636 -----
----- Voxel7637 -----
----- Voxel7638 -----
----- Voxel7639 -----
----- Voxel7640 -----
----- Voxel7641 -----
----- Voxel7642 -----
----- Voxel7643 -----
----- Voxel7644 -----
----- Voxel7645 -----
----- Voxel7646 -----
----- Voxel7647 -----
----- Voxel7648 -----
----- Voxel7649 -----
----- Voxel7650 -----
----- Voxel7651 -----
----- Voxel7652 -----
----- Voxel7653 -----
----- Voxel7654 -----
----- Voxel7655 -----
----- Voxel7656 -----
----- Voxel7657 -----
----- Voxel7658 -----
----- Voxel7659 -----
----- Voxel7660 -----
----- Voxel7661 -----
----- Voxel7662 -----
----- Voxel7663 -----
----- Voxel7664 -----
----- Voxel7665 -----
----- Voxel7666 -----
----- Voxel7667 -----
----- Voxel7668 -----
----- Voxel7669 -----
----- Voxel7670 -----
----- Voxel7671 -----
----- Voxel7672 -----
----- Voxel7673 -----
----- Voxel7674 -----
----- Voxel7675 -----
----- Voxel7676 -----
----- Voxe

----- Voxel8005 -----
----- Voxel8006 -----
----- Voxel8007 -----
----- Voxel8008 -----
----- Voxel8009 -----
----- Voxel8010 -----
----- Voxel8011 -----
----- Voxel8012 -----
----- Voxel8013 -----
----- Voxel8014 -----
----- Voxel8015 -----
----- Voxel8016 -----
----- Voxel8017 -----
----- Voxel8018 -----
----- Voxel8019 -----
----- Voxel8020 -----
----- Voxel8021 -----
----- Voxel8022 -----
----- Voxel8023 -----
----- Voxel8024 -----
----- Voxel8025 -----
----- Voxel8026 -----
----- Voxel8027 -----
----- Voxel8028 -----
----- Voxel8029 -----
----- Voxel8030 -----
----- Voxel8031 -----
----- Voxel8032 -----
----- Voxel8033 -----
----- Voxel8034 -----
----- Voxel8035 -----
----- Voxel8036 -----
----- Voxel8037 -----
----- Voxel8038 -----
----- Voxel8039 -----
----- Voxel8040 -----
----- Voxel8041 -----
----- Voxel8042 -----
----- Voxel8043 -----
----- Voxel8044 -----
----- Voxel8045 -----
----- Voxel8046 -----
----- Voxel8047 -----
----- Voxel8048 -----
----- Voxel8049 -----
----- Voxe

----- Voxel8379 -----
----- Voxel8380 -----
----- Voxel8381 -----
----- Voxel8382 -----
----- Voxel8383 -----
----- Voxel8384 -----
----- Voxel8385 -----
----- Voxel8386 -----
----- Voxel8387 -----
----- Voxel8388 -----
----- Voxel8389 -----
----- Voxel8390 -----
----- Voxel8391 -----
----- Voxel8392 -----
----- Voxel8393 -----
----- Voxel8394 -----
----- Voxel8395 -----
----- Voxel8396 -----
----- Voxel8397 -----
----- Voxel8398 -----
----- Voxel8399 -----
----- Voxel8400 -----
----- Voxel8401 -----
----- Voxel8402 -----
----- Voxel8403 -----
----- Voxel8404 -----
----- Voxel8405 -----
----- Voxel8406 -----
----- Voxel8407 -----
----- Voxel8408 -----
----- Voxel8409 -----
----- Voxel8410 -----
----- Voxel8411 -----
----- Voxel8412 -----
----- Voxel8413 -----
----- Voxel8414 -----
----- Voxel8415 -----
----- Voxel8416 -----
----- Voxel8417 -----
----- Voxel8418 -----
----- Voxel8419 -----
----- Voxel8420 -----
----- Voxel8421 -----
----- Voxel8422 -----
----- Voxel8423 -----
----- Voxe

KeyboardInterrupt: 

In [19]:
# もしデータセットの整形でサイズエラーが出た時は以下で.niiファイル読み込みしてサイズを確認する
# nii0 = nib.load('../../Data_mri/tappingState-2fe/20181029rn/64ch/wmaskAll.nii')
# img0 = nii0.get_data()
# img0.shape

(79, 95, 79)